In [45]:
# import ocean row module
from mod_ocean_row import ocean_row, parameter_sweeper
import xarray as xr
# # reload package
import importlib
import mod_ocean_row
importlib.reload(mod_ocean_row)
from mod_ocean_row import ocean_row

In [40]:
def create_config():
    # create dictionary of starting location
    # Dsloc = {'start_name':'Fremantle'    ,'start_lat':-32.056946,'start_lon':115.743889}
    # Dsloc = {'start_name':'Geraldton'    ,'start_lat':-28.782387,'start_lon':114.607513} 
    Dsloc = {'start_name':'Kalbarri'     ,'start_lat':-27.711053,'start_lon':114.164536} 
    # Dsloc = {'start_name':'Carnarvon'    ,'start_lat':-24.883717,'start_lon':113.656956} 
    # Dsloc = {'start_name':'Exmouth'      ,'start_lat':-21.930724,'start_lon':114.122389} 
    # Dsloc = {'start_name':'Broome'       ,'start_lat':-17.951221,'start_lon':122.244327} 

    # create dictionary of destination location
    Deloc = {'stop_name':'Dar es Salaam' ,'stop_lat':-6.766305  ,'stop_lon':39.195973  } # create dictionary of destination location

    # date and time
    tod_0     = '2018-05-01 03:00:00' # starting datetime
    tod_start = ['06:00:00','14:00:00'] # start rowing [time of day]
    tod_stop  = ['12:00:00','20:00:00'] # stop rowing [time of day]
    Dtime     = {'tod_0':tod_0,'tod_start':tod_start,'tod_stop':tod_stop} # create dictionary

    # rowing properties
    m     = 670 # wiegth [kg]

    # convert m/s to kts
    tokts = 1.94384449 # convert m/s to kts

    # rowing speed perfect conditions
    vperf = 1.7/tokts # boat velocity in perfect conditions [m/s]

    # water friction - boat
    rhow  = 1024 # density water [kg/m^3]
    Aw    = 15 # frontal area exposed to water
    Cdw   = 0.005 # friction coef. water 
    Gw    = rhow*Cdw*Aw/2 # water friction constant

    # water friction - anchor
    GwA   = 50*Gw # water friction constant sea anchor

    # friction ratio boat
    G     = .01 # ratio of air friction cosntant to water friction constant (Ga/Gw)

    # air friction 
    Ga    = G*Gw # air friction constant

    Drow_prop = {'m':m,'vperf':vperf,'Gw':Gw,'GwA':GwA,'Ga':Ga} # create dictionary of all rowing properties

    # combine all configurations
    mcf = Dsloc | Deloc | Drow_prop | Dtime # model configuration (as dictionary)
    return mcf

def load_hindcast():

    # air velocity (wind)
    dsa0 = xr.open_dataset(r'./Data/CERSAT-GLO-REP_WIND_L4-OBS_FULL_TIME_SERIE_1648775425193.nc').load()
    dsa  = xr.Dataset(
        data_vars=dict(
            # correct wind dataset (wrong direction of winds)
            ua  = (["time", "lat", "lon"], dsa0.eastward_wind.isel(depth=0).values),
            va  = (["time", "lat", "lon"], dsa0.northward_wind.isel(depth=0).values)),
        coords=dict(
            lon  = dsa0.longitude.values,
            lat  = dsa0.latitude.values,
            time = dsa0.time.values))

    # water velocity (currents)
    dsw0 = xr.open_dataset(r'./Data/nemo_monthly_mean.nc').load()
    dsw  = xr.Dataset(
        data_vars=dict(
            uw  = (["time", "lat", "lon"], dsw0.uo.isel(depth=0).values),
            vw  = (["time", "lat", "lon"], dsw0.vo.isel(depth=0).values),
            Tw  = (["time", "lat", "lon"], dsw0.thetao.isel(depth=0).values)),
        coords=dict(
            lon  = dsw0.longitude.values,
            lat  = dsw0.latitude.values,
            time = dsw0.time.values))

    # wave heights 
    dswh0 = xr.open_dataset(r'./Data/cmems_obs-wave_glo_phy-swh_my_multi-l4-2deg_P1D_1648776708969.nc').load()
    dswh  = xr.Dataset(
        data_vars=dict(
            # correct wind dataset (wrong direction of winds)
            H_day_mean = (["time", "lat", "lon"], dswh0.VAVH_DAILY_MEAN.values),
            H_day_max  = (["time", "lat", "lon"], dswh0.VAVH_DAILY_MAX.values)),
        coords=dict(
            lon  = dswh0.longitude.values,
            lat  = dswh0.latitude.values,
            time = dswh0.time.values))

    hc = {'dsa':dsa,'dsw':dsw,'dswh':dswh} # create single dictonary of all datasets
    return hc




In [41]:

hc = load_hindcast()

In [46]:
mcf = create_config()
row = ocean_row(mcf)
row.mcts(hc, 90, 5, 200000, 10000000)

(array([[-3.06999993,  1.8499999 ]]), array([[0., 0.]]))

In [ ]:
(